# 1. úloha

## 1.1
TODO pridat zvolenu parametrizaciu Weibullovho rozdelenia, log-vierohodnostnu funkciu a parcialne derivacie

In [27]:
import scipy
import openpyxl
import pandas as pd
import numpy as np

## 1.2

In [28]:
path = "Data_2024.xlsx"
wb = openpyxl.load_workbook(path)
sheet = wb["Data_věrohodnost"]


data = []

uncensored_data = []
censored_data = []

for row in sheet.iter_rows(min_row=2, max_col=2, values_only=True):
    data.append(row[1])
    
    if(row[0] == 0):
        uncensored_data.append(row[1])
    else:
        censored_data.append(row[1])
    
uncensored_data = np.array(uncensored_data)
censored_data = np.array(censored_data)



In [29]:
def log_likelihood(params, uncensored_data, censored_data):
    k, lambd = params
    
    ll_uncensored = np.sum(np.log(k/lambd) + np.log((uncensored_data/lambd)**(k-1)) - (uncensored_data/lambd)**k)
    
    
    ll_censored = np.sum(-(censored_data / lambd)**k)

    return -(ll_uncensored + ll_censored)

def log_likelihood_exp(params, uncensored_data, censored_data):
    lambd = params

    ll_uncensored = np.sum(np.log(1/lambd) + np.log((uncensored_data/lambd)**(1-1)) - (uncensored_data/lambd)**1)
    
    
    ll_censored = np.sum(-(censored_data / lambd)**1)

    return -(ll_uncensored + ll_censored)



In [ ]:
import scipy.optimize

initial_guess = [1.5, np.mean(data)]

result = scipy.optimize.minimize(fun=log_likelihood, x0=initial_guess, args=(uncensored_data, censored_data), method='BFGS')

k_opt, lambd_opt = result.x
print(f"est. params: k = {k_opt:.4f}, lambda = {lambd_opt:.4f}")

## 1.3

In [ ]:
initial_guess_exp = [np.mean(data)]

result_exp = scipy.optimize.minimize(fun=log_likelihood_exp, x0=initial_guess_exp, args=(uncensored_data, censored_data), method='BFGS')

L1 = result.fun

L2 = result_exp.fun

Stat = 2 * (L2 - L1)

print(Stat)



Doplnok kritického oboru: <0; 5.024>
592.39 nepatrí do doplnku kritického oboru, teda zamietame H0: Exponenciálne rozdelenie je postačujúce
Prijímame H1: Exponenciálne rozdelenie nie je postačujúce

## 1.4

In [ ]:
median = lambd_opt*(np.log(2))**(1/k_opt) # TODO check, guy on discord has 6.9, I have 7.0
tencentil = lambd_opt*(-np.log(1-0.1))**(1/k_opt)

print("median:", median)
print("tencentil:", tencentil)
